In [12]:
import os
from PIL import Image
import numpy as np
from sklearn.preprocessing import LabelEncoder
from tensorflow.keras.utils import to_categorical

# Define the path to the directory
directory_path = r'F:\OfficeHomeDataset_10072016\OfficeHomeDataset_10072016\Real World'

# Initialize empty lists for images and labels
X = []
Y = []

# Initialize LabelEncoder
label_encoder = LabelEncoder()

# Create a dictionary to map folder names to unique labels
folder_to_label = {}

# Loop through subdirectories (classes) and assign unique labels
for index, class_folder in enumerate(os.listdir(directory_path)):
    class_path = os.path.join(directory_path, class_folder)

    # Store the mapping of folder name to unique label
    folder_to_label[class_folder] = index

    for image_file in os.listdir(class_path):
        try:
            if image_file.endswith('.jpg'):
                image_path = os.path.join(class_path, image_file)

                # Load image, convert to RGB and resize
                img = Image.open(image_path).convert('RGB')
                img = img.resize((32, 32))
                img_array = np.array(img)

                # Append image and label to lists
                X.append(img_array)
                Y.append(index)  # Use the unique label for this folder
        except Exception as e:
            print(f"Error processing image file {image_file}: {e}")

# Convert lists to numpy arrays
X = np.array(X)

# Encode labels using one-hot encoding
Y = to_categorical(Y, num_classes=65)  # 65 different folders

print(f'X shape: {X.shape}')
print(f'Y shape: {Y.shape}')


X shape: (4357, 32, 32, 3)
Y shape: (4357, 65)


In [13]:
from sklearn.model_selection import train_test_split

# Split the data into train and test sets
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.2, random_state=42)

print(f'X_train shape: {X_train.shape}')
print(f'Y_train shape: {Y_train.shape}')
print(f'X_test shape: {X_test.shape}')
print(f'Y_test shape: {Y_test.shape}')


X_train shape: (3485, 32, 32, 3)
Y_train shape: (3485, 65)
X_test shape: (872, 32, 32, 3)
Y_test shape: (872, 65)


In [19]:
from tensorflow import keras
model = keras.models.load_model(r"C:\Users\shaif\Downloads\office_real_32_model.h5")

from tensorflow.keras.applications import ResNet50
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D
    
num_classes = 65  # Update with the actual number of classes in your target data
x = model.layers[-4].output  # Access the last 4th layer from the end
x = GlobalAveragePooling2D()(x)
output = Dense(num_classes, activation='softmax')(x)

# Create the new model with the updated head
new_model = keras.models.Model(inputs=model.input, outputs=output)
new_model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
# Train the model with early stopping
new_model.fit(X_train, Y_train,validation_split = 0.2, batch_size=32, epochs=100)

Epoch 1/100
88/88 [==============================] - 13s 77ms/step - loss: 5.1287 - accuracy: 0.0603 - val_loss: 179.0523 - val_accuracy: 0.0129
Epoch 2/100
88/88 [==============================] - 4s 44ms/step - loss: 4.8280 - accuracy: 0.0714 - val_loss: 308.3452 - val_accuracy: 0.0287
Epoch 3/100
88/88 [==============================] - 4s 43ms/step - loss: 4.6448 - accuracy: 0.0757 - val_loss: 6.7680 - val_accuracy: 0.0760
Epoch 4/100
88/88 [==============================] - 4s 43ms/step - loss: 4.4730 - accuracy: 0.0922 - val_loss: 12.0361 - val_accuracy: 0.0746
Epoch 5/100
88/88 [==============================] - 4s 49ms/step - loss: 4.4156 - accuracy: 0.1119 - val_loss: 1771.8322 - val_accuracy: 0.0445
Epoch 6/100
88/88 [==============================] - 4s 50ms/step - loss: 4.6779 - accuracy: 0.0717 - val_loss: 26.5750 - val_accuracy: 0.0631
Epoch 7/100
88/88 [==============================] - 4s 50ms/step - loss: 4.6362 - accuracy: 0.0671 - val_loss: 6.3638 - val_accuracy: 0.0

88/88 [==============================] - 5s 52ms/step - loss: 0.6603 - accuracy: 0.8529 - val_loss: 5.2812 - val_accuracy: 0.1923
Epoch 59/100
88/88 [==============================] - 5s 54ms/step - loss: 0.5904 - accuracy: 0.8956 - val_loss: 5.7067 - val_accuracy: 0.1894
Epoch 60/100
88/88 [==============================] - 5s 52ms/step - loss: 0.5546 - accuracy: 0.8809 - val_loss: 5.9244 - val_accuracy: 0.1908
Epoch 61/100
88/88 [==============================] - 5s 54ms/step - loss: 0.6485 - accuracy: 0.8655 - val_loss: 5.5823 - val_accuracy: 0.1937
Epoch 62/100
88/88 [==============================] - 5s 54ms/step - loss: 0.6199 - accuracy: 0.8766 - val_loss: 6.3373 - val_accuracy: 0.2080
Epoch 63/100
88/88 [==============================] - 5s 54ms/step - loss: 0.5095 - accuracy: 0.9039 - val_loss: 5.9989 - val_accuracy: 0.1980
Epoch 64/100
88/88 [==============================] - 5s 53ms/step - loss: 0.5146 - accuracy: 0.9017 - val_loss: 5.9329 - val_accuracy: 0.1994
Epoch 65/100

In [20]:
import numpy as np
from sklearn.metrics import accuracy_score, f1_score

# Assuming you have trained a model and obtained predicted probabilities on x_test
y_pred_prob = new_model.predict(X_test)

# Convert predicted probabilities to predicted labels
y_pred = np.argmax(y_pred_prob, axis=1)

# Convert y_test to predicted labels format
y_test_labels = np.argmax(Y_test, axis=1)

# Calculate accuracy
accuracy = accuracy_score(y_test_labels, y_pred)
print("Accuracy:", accuracy)

# Calculate F1 score (micro-average)
f1_micro = f1_score(y_test_labels, y_pred, average='micro')
print("F1 Score (Micro):", f1_micro)

28/28 [==============================] - 2s 30ms/step
Accuracy: 0.2121559633027523
F1 Score (Micro): 0.2121559633027523


In [21]:
from tensorflow.keras.applications import ResNet50
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D

# Create the ResNet model without the top (fully connected) layers
base_model = ResNet50(weights='imagenet', include_top=False, input_shape=(32, 32, 3))
    
# Add the top layers for classification
model = Sequential()
model.add(base_model)
model.add(GlobalAveragePooling2D())
model.add(Dense(65, activation='softmax'))

# Compile the model
model.compile(optimizer=Adam(lr=0.001), loss='categorical_crossentropy', metrics=['accuracy'])
# Train the model
model.fit(X_train, Y_train, batch_size=32, epochs=100, validation_split = .2)

Epoch 1/100


C:\Users\shaif\anaconda3\envs\tens\lib\site-packages\keras\optimizers\optimizer_v2\adam.py:110: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super(Adam, self).__init__(name, **kwargs)


88/88 [==============================] - 12s 67ms/step - loss: 4.7903 - accuracy: 0.0714 - val_loss: 8.1750 - val_accuracy: 0.0531
Epoch 2/100
88/88 [==============================] - 5s 56ms/step - loss: 4.2607 - accuracy: 0.0994 - val_loss: 16.2618 - val_accuracy: 0.0187
Epoch 3/100
88/88 [==============================] - 5s 59ms/step - loss: 4.5316 - accuracy: 0.0775 - val_loss: 17.3926 - val_accuracy: 0.0344
Epoch 4/100
88/88 [==============================] - 5s 56ms/step - loss: 4.4399 - accuracy: 0.0864 - val_loss: 8.6135 - val_accuracy: 0.0588
Epoch 5/100
88/88 [==============================] - 5s 61ms/step - loss: 4.4135 - accuracy: 0.0907 - val_loss: 4.8565 - val_accuracy: 0.0617
Epoch 6/100
88/88 [==============================] - 5s 60ms/step - loss: 4.0357 - accuracy: 0.1123 - val_loss: 4.2735 - val_accuracy: 0.0732
Epoch 7/100
88/88 [==============================] - 5s 55ms/step - loss: 4.0548 - accuracy: 0.1374 - val_loss: 9.9162 - val_accuracy: 0.0961
Epoch 8/100
88/

88/88 [==============================] - 6s 71ms/step - loss: 3.3191 - accuracy: 0.2493 - val_loss: 6.0890 - val_accuracy: 0.0717
Epoch 59/100
88/88 [==============================] - 6s 73ms/step - loss: 2.1240 - accuracy: 0.4297 - val_loss: 4.7091 - val_accuracy: 0.1750
Epoch 60/100
88/88 [==============================] - 6s 70ms/step - loss: 1.4768 - accuracy: 0.6148 - val_loss: 4.9022 - val_accuracy: 0.1750
Epoch 61/100
88/88 [==============================] - 6s 73ms/step - loss: 1.1855 - accuracy: 0.7048 - val_loss: 5.1160 - val_accuracy: 0.2138
Epoch 62/100
88/88 [==============================] - 7s 76ms/step - loss: 0.9104 - accuracy: 0.7669 - val_loss: 4.8529 - val_accuracy: 0.2080
Epoch 63/100
88/88 [==============================] - 6s 73ms/step - loss: 0.7236 - accuracy: 0.8189 - val_loss: 5.2127 - val_accuracy: 0.2152
Epoch 64/100
88/88 [==============================] - 6s 68ms/step - loss: 0.6247 - accuracy: 0.8418 - val_loss: 5.2941 - val_accuracy: 0.2080
Epoch 65/100

In [22]:
import numpy as np
from sklearn.metrics import accuracy_score, f1_score

# Assuming you have trained a model and obtained predicted probabilities on x_test
y_pred_prob = model.predict(X_test)

# Convert predicted probabilities to predicted labels
y_pred = np.argmax(y_pred_prob, axis=1)

# Convert y_test to predicted labels format
y_test_labels = np.argmax(Y_test, axis=1)

# Calculate accuracy
accuracy = accuracy_score(y_test_labels, y_pred)
print("Accuracy:", accuracy)

# Calculate F1 score (micro-average)
f1_micro = f1_score(y_test_labels, y_pred, average='micro')
print("F1 Score (Micro):", f1_micro)

28/28 [==============================] - 2s 30ms/step
Accuracy: 0.21330275229357798
F1 Score (Micro): 0.21330275229357798
